In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from sklearn.metrics import f1_score, recall_score, precision_score

In [23]:
df_train_1 = pd.read_csv("https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/df_train_0.csv", header=0, index_col=0)
df_train_2 = pd.read_csv("https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/df_train_1.csv", header=0, index_col=0)

df = pd.concat([df_train_1, df_train_2], axis=0, ignore_index=True)

In [24]:
X = df.iloc[:,93]
Y = df[['Dos','normal','Probe','R2L','U2R']]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [ ]:
#MLP
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Dense(units=50, activation='relu'))
mlp.add(tf.keras.layers.Dense(units=5, activation='softmax'))
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = mlp.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

test_results = mlp.evaluate(X_test, y_test, verbose=1)

pred = mlp.predict(X_test)
for j in range(0,pred.shape[1]):
  for i in range(0,pred.shape[0]):
    pred[i][j] = int(round(pred[i][j]))
pred_df = pd.DataFrame(pred,columns=y_test.columns)

print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
print("Recall Score - ",recall_score(y_test,pred_df.astype('uint8'),average='micro'))
print("F1 Score - ",f1_score(y_test,pred_df.astype('uint8'),average='micro'))
print("Precision Score - ",precision_score(y_test,pred_df.astype('uint8'),average='micro'))

In [ ]:
mlp.summary()

In [ ]:
plot_model(mlp, show_shapes=True)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Plot of accuracy vs epoch for train and test dataset")
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Plot of loss vs epoch for train and test dataset")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()